In [1]:
import pandas as pd

path_to_train = './data/train.csv'
path_to_test = './data/test.csv'

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type', 'is_listened'])
test = pd.read_csv(path_to_test, encoding='utf-8', usecols=['user_id', 'ts_listen', 'listen_type'])
test['is_listened'] = 0 # просто щоб зібрати, ми будемо використовувати попередні значення

df = pd.concat([train, test], ignore_index=True)
df.tail()

,ts_listen,listen_type,user_id,is_listened
7578747,1479664431,1,12641,0
7578748,1480358099,1,10055,0
7578749,1480616853,1,1029,0
7578750,1479644510,1,4630,0
7578751,1480491885,1,6467,0


In [2]:
df = df.sort_values(by=['user_id', 'ts_listen'], axis=0)
df.tail()

,ts_listen,listen_type,user_id,is_listened
7575803,1478016326,1,19915,0
569368,1477512816,0,19916,1
7563826,1478017434,1,19916,0
4791398,1477926978,0,19917,1
7565747,1478009685,1,19917,0


In [3]:
df.head()

,ts_listen,listen_type,user_id,is_listened
682493,1477939775,0,0,1
4454939,1477982531,0,0,0
5354523,1477982561,0,0,0
5899458,1477982582,0,0,1
4716574,1477982792,0,0,1


In [41]:
# counter_flow = 0
# counter_listened = 0
# prev_user = -1
# prev_flow = -1
# prev_listened = -1

# def inline_iterate(row):
#     #row = tp[1]
#     global counter_flow, counter_listened, prev_user, prev_flow, prev_listened
    
#     ret_value = (counter_flow, counter_listened)
    
#     if(row['listen_type'] != prev_flow):
#         prev_flow = row['listen_type']
#         counter_flow = 0
    
#     if(row['is_listened'] != prev_listened):
#         prev_listened = row['is_listened']
#         counter_listened = 0
        
#     if(row['user_id'] != prev_user):
#         prev_user = row['user_id']
#         counter_flow = 0
#         counter_listened = 0
        
#     counter_flow += 1
#     counter_listened += 1
    
#     return counter_flow, counter_listened

#inline = [inline_iterate(row) for index,row in df.iterrows()]
#inline = list(map(inline_iterate, df.iterrows()))
#inline

#df['cumsum_listen_type'] = [i[0] for i in inline]
#df['cumsum_is_listened'] = [i[1] for i in inline]

#df['cumsum_listen_type_flow'] = df.apply(lambda row: row['cumsum_listen_type'] if row['listen_type'] == 1 else 0, axis=1)
#df['cumsum_listen_type_not_flow'] = df.apply(lambda row: row['cumsum_listen_type'] if row['listen_type'] == 0 else 0, axis=1)

#df['cumsum_is_listened_1'] = df.apply(lambda row: row['cumsum_is_listened'] if row['is_listened'] == 1 else 0, axis=1)#
#df['cumsum_is_listened_0'] = df.apply(lambda row: row['cumsum_is_listened'] if row['is_listened'] == 0 else 0, axis=1)

In [13]:
df['cumsum_listen_type'] = (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1).shift()

df['cumsum_listen_type_1'] = (df['listen_type'] * (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

df['cumsum_listen_type_0'] = (abs(1-df['listen_type']) * (df['listen_type'].groupby((
                (df[['listen_type','user_id']].shift() != df[['listen_type','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

fi = df.groupby('user_id', as_index=False).apply(lambda x: x.index[0])
df.ix[fi, 'cumsum_listen_type_1'] = 0
df.ix[fi, 'cumsum_listen_type_0'] = 0
df.ix[fi, 'cumsum_listen_type'] = 0

df[['user_id', 'listen_type', 'cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0']].tail(10)

,user_id,listen_type,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0
2549309,19913,0,0.0,0.0,0.0
7575054,19913,1,1.0,0.0,1.0
2857394,19914,0,0.0,0.0,0.0
7573591,19914,1,1.0,0.0,1.0
4242361,19915,1,0.0,0.0,0.0
7575803,19915,1,1.0,1.0,0.0
569368,19916,0,0.0,0.0,0.0
7563826,19916,1,1.0,0.0,1.0
4791398,19917,0,0.0,0.0,0.0
7565747,19917,1,1.0,0.0,1.0


In [14]:
df[['user_id', 'listen_type', 'cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0']].head(10)

,user_id,listen_type,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0
682493,0,0,0.0,0.0,0.0
4454939,0,0,1.0,0.0,1.0
5354523,0,0,2.0,0.0,2.0
5899458,0,0,3.0,0.0,3.0
4716574,0,0,4.0,0.0,4.0
2859238,0,0,5.0,0.0,5.0
3796053,0,0,6.0,0.0,6.0
1209271,0,0,7.0,0.0,7.0
6888438,0,0,8.0,0.0,8.0
6714892,0,0,9.0,0.0,9.0


In [15]:
df['cumsum_is_listened'] = (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1).shift()

df['cumsum_is_listened_1'] = (df['is_listened'] * (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

df['cumsum_is_listened_0'] = (abs(1-df['is_listened']) * (df['is_listened'].groupby((
                (df[['is_listened','user_id']].shift() != df[['is_listened','user_id']]).any(axis=1)
            ).cumsum()).cumcount() + 1)).shift()

fi = df.groupby('user_id', as_index=False).apply(lambda x: x.index[0])
df.ix[fi, 'cumsum_is_listened_1'] = 0
df.ix[fi, 'cumsum_is_listened_0'] = 0
df.ix[fi, 'cumsum_is_listened'] = 0

df[['user_id', 'is_listened', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']].tail(10)

,user_id,is_listened,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
2549309,19913,1,0.0,0.0,0.0
7575054,19913,0,1.0,1.0,0.0
2857394,19914,1,0.0,0.0,0.0
7573591,19914,0,1.0,1.0,0.0
4242361,19915,1,0.0,0.0,0.0
7575803,19915,0,1.0,1.0,0.0
569368,19916,1,0.0,0.0,0.0
7563826,19916,0,1.0,1.0,0.0
4791398,19917,1,0.0,0.0,0.0
7565747,19917,0,1.0,1.0,0.0


In [16]:
df[['user_id', 'is_listened', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']].head(10)

,user_id,is_listened,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
682493,0,1,0.0,0.0,0.0
4454939,0,0,1.0,1.0,0.0
5354523,0,0,1.0,0.0,1.0
5899458,0,1,2.0,0.0,2.0
4716574,0,1,1.0,1.0,0.0
2859238,0,1,2.0,2.0,0.0
3796053,0,1,3.0,3.0,0.0
1209271,0,1,4.0,4.0,0.0
6888438,0,1,5.0,5.0,0.0
6714892,0,1,6.0,6.0,0.0


In [17]:
df = df.sort_index()
df.tail()

,ts_listen,listen_type,user_id,is_listened,cumsum_listen_type,cumsum_listen_type_1,cumsum_listen_type_0,cumsum_is_listened,cumsum_is_listened_1,cumsum_is_listened_0
7578747,1479664431,1,12641,0,21.0,0.0,21.0,1.0,1.0,0.0
7578748,1480358099,1,10055,0,3.0,3.0,0.0,66.0,66.0,0.0
7578749,1480616853,1,1029,0,5.0,5.0,0.0,8.0,8.0,0.0
7578750,1479644510,1,4630,0,19.0,0.0,19.0,18.0,18.0,0.0
7578751,1480491885,1,6467,0,316.0,316.0,0.0,3.0,3.0,0.0


In [1]:
path_to_test_cumsum = './data/features/cumsum.test.csv'
path_to_train_cumsum = './data/features/cumsum.train.csv'

In [19]:
test_cumsum = df.iloc[range(len(train), len(train)+len(test))][['cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']]
test_cumsum.to_csv(path_to_test_cumsum, encoding='utf-8', index=False)

train_cumsum = df.iloc[range(len(train))][['cumsum_listen_type', 'cumsum_listen_type_1', 'cumsum_listen_type_0', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']]
train_cumsum.to_csv(path_to_train_cumsum, encoding='utf-8', index=False)

Спробуємо, чи є покращення?

In [2]:
import validation as v
import pandas as pd

path_to_train = './data/train.csv'
#path_to_test = './data/test.csv'

train = pd.read_csv(path_to_train, encoding='utf-8')
#test = pd.read_csv(path_to_test, encoding='utf-8')

train = pd.concat([train, pd.read_csv(path_to_train_cumsum, encoding='utf-8')], axis=1)

path_to_folds = './data/lightgbm'
v.generate_validation_folds(train, path_to_folds)

Fold 1


In [6]:
import model_lgbm as mlgbm

cols = ['genre_id', 'media_id', 'album_id', 'context_type',
       'release_date', 'platform_name', 'platform_family', 'media_duration',
       'listen_type', 'user_gender', 'user_id', 'artist_id', 'user_age', 'cumsum_listen_type', 'cumsum_listen_type_1', 
        'cumsum_listen_type_0', 'cumsum_is_listened', 'cumsum_is_listened_1', 'cumsum_is_listened_0']

params = {
    'application':'binary',
    'num_leaves': 31,
    'max_depth': 20,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'max_bin': 200,
    'metric': 'auc',
    'verbose': 1
    
}

In [ ]:
evs = mlgbm.crossvalidate_model(path_to_folds,cols,params)

[1]	valid_0's auc: 0.695661
[2]	valid_0's auc: 0.695656
[3]	valid_0's auc: 0.695426
[4]	valid_0's auc: 0.695119
[5]	valid_0's auc: 0.69294
[6]	valid_0's auc: 0.69366
[7]	valid_0's auc: 0.694481
[8]	valid_0's auc: 0.694778
[9]	valid_0's auc: 0.695228
[10]	valid_0's auc: 0.695384
[11]	valid_0's auc: 0.695497
[12]	valid_0's auc: 0.696108
[13]	valid_0's auc: 0.696197
[14]	valid_0's auc: 0.696958
[15]	valid_0's auc: 0.697207
[16]	valid_0's auc: 0.697869
[17]	valid_0's auc: 0.698424
[18]	valid_0's auc: 0.699169
[19]	valid_0's auc: 0.699225
[20]	valid_0's auc: 0.699313
[21]	valid_0's auc: 0.700258
[22]	valid_0's auc: 0.700268
[23]	valid_0's auc: 0.700274
[24]	valid_0's auc: 0.700178
[25]	valid_0's auc: 0.700904
[26]	valid_0's auc: 0.700919
[27]	valid_0's auc: 0.700975
[28]	valid_0's auc: 0.701025
[29]	valid_0's auc: 0.701827
[30]	valid_0's auc: 0.702242
[31]	valid_0's auc: 0.702202
[32]	valid_0's auc: 0.702208
[33]	valid_0's auc: 0.703889
[34]	valid_0's auc: 0.704719
[35]	valid_0's auc: 0.704

In [15]:
import numpy as np
np.mean(evs)

0.69788057877479892

In [3]:
path_to_sample = './data/sample_submission_kaggle.csv'
path_to_submission = './data/submission.csv'
path_to_test = './data/test.csv'
path_to_feature_test = './data/features/inline.test.csv'

test = pd.read_csv(path_to_test, encoding='utf-8')
test = pd.concat([test, pd.read_csv(path_to_feature_test, encoding='utf-8')], axis=1)

test.head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,inline,inline_flow,inline_not_flow
0,0,50,1478104371,683078,82356,1,20021008,0,0,542,1,0,17698,2076,30,10,10,0
1,1,2744,1479317140,876497,99692,1,19851231,0,0,307,1,0,10525,26,28,1,1,0
2,2,2744,1479546361,876497,99692,1,19851231,0,0,307,1,0,8716,26,27,10,10,0
3,3,2744,1478457729,876500,99692,1,19851231,2,1,265,1,0,5443,26,30,1,1,0
4,4,2744,1480448560,876504,99692,1,19851231,2,1,356,1,0,7600,26,29,10,10,0


In [4]:
fs = mlgbm.create_submission(train, test, path_to_sample, params, cols, path_to_submission)

In [5]:
fscore = list(zip(cols, fs))
fscore.sort(key=lambda x: x[1])
fscore.reverse()
fscore

[('user_id', 7563),
 ('user_age', 4241),
 ('context_type', 2736),
 ('artist_id', 2330),
 ('release_date', 2000),
 ('media_duration', 1889),
 ('media_id', 1621),
 ('genre_id', 1608),
 ('platform_name', 1398),
 ('album_id', 1194),
 ('user_gender', 1167),
 ('inline', 992),
 ('platform_family', 948),
 ('inline_flow', 313),
 ('inline_not_flow', 0),
 ('listen_type', 0)]